## OpenFeign
OpenFeign utilizes a service contract and creates service client implementation using it. As a simplistic example:

In [ ]:
// This interface defines service contract with Github
interface GitHub {
  @RequestLine("GET /repos/{owner}/{repo}/contributors")
  List<Contributor> contributors(@Param("owner") String owner, @Param("repo") String repo);
}

class Contributer {
  String login;
  int contributions;
}

public class Demo {
  public static void main(String... args) {
    GitHub github = Feign.builder()
                         .decoder(new GsonDecoder())
                         .target(GitHub.class, "https://api.github.com");

    // Fetch and print a list of the contributors to this library.
    List<Contributor> contributors = github.contributors("OpenFeign", "feign");
    for (Contributor contributor : contributors) {
      System.out.println(contributor.login + " (" + contributor.contributions + ")");
    }
  }
}

### Contract
`Contract` defines the contract between the interface and how the underlying client should work. The default `Contract` bundled with Feign utilises annotations like `@RequestLine`, `@Param`, `@Headers`, `@QueryMap`, `@HeaderMap` and `@Body`. Sample interface demonstrating various options:

In [ ]:
interface MyApi {
  @RequestLine("GET /api/documents/{contentType}")
  @Headers("Accept: {contentType}")
  String getDocumentByType(@Param("contentType") String type);
    
  @RequestLine("POST /")
  void postData(@HeaderMap Map<String, Object> headerMap);
  
  @RequestLine("GET /find")
  Book findBook(@QueryMap Map<String, Object> queryMap);
    
  @RequestLine("POST /LOGIN")
  @Headers("Content-Type: application/json")
  // json curly braces must be escaped!
  @Body("%7B\"user_name\": \"{user_name}\", \"password\": \"{password}\"%7D")
  void LOGIN(@Param("user_name") String user, @Param("password") String password);
}

Users using JAX-RS would prefer using JAX-RS conforming service contract. In that case, we need to specify contract while building the Feign client:

In [ ]:
interface GitHub {
  @GET @Path("/repos/{owner}/{repo}/contributors")
  List<Contributor> contributors(@PathParam("owner") String owner, @PathParam("repo") String repo);
}

public class Example {
  public static void main(String[] args) {
    GitHub github = Feign.builder()
                       .contract(new JAXRSContract())
                       .target(GitHub.class, "https://api.github.com");
  }
}

A `Contract` implementation defines what annotations and values are valid on interfaces. It is called to parse the methods in the class that are linked to HTTP requests.

### Decoder
Decodes an HTTP response into a single object of the given type. Invoked when `Response.status()` is in the 2xx range and the return type is neither void nor `Response`.

In [ ]:
import java.io.IOException;
import java.lang.reflect.Type;
import feign.FeignException;
import feign.Response;

public interface Decoder {
  Object decode(Response response, Type type) throws IOException, DecodeException, FeignException;
}

As an example, `GsonDecoder` works on JSON outputs:

In [ ]:
GitHub github = Feign.builder()
                     .decoder(new GsonDecoder())
                     .target(GitHub.class, "https://api.github.com");

### Encoder
Encodes an object into an HTTP request body. By configuring an Encoder, we can send a type-safe request body. 

In [ ]:
// instead of a String representing request body
interface LoginClient {
  @RequestLine("POST /")
  @Headers("Content-Type: application/json")
  void login(String content);
}

// we can have a Java class representing the same
interface LoginClient {
  @RequestLine("POST /")
  @Headers("Content-Type: application/json")
  void login(LoginCredentials credentials);
}

Some examples include `SpringEncoder`, `SpringFormEncoder`, `JacksonEncoder`.

In [ ]:
public interface Encoder {
  void encode(Object object, Type bodyType, RequestTemplate template) throws EncodeException;
}

### Client
Feign lets us specify the HTTP client to use such as OkHttpClient or Apache HTTP Client. Any client that we intend to use must conform to:

In [ ]:
// Option class contains value for connect timeout, read timeout and
// whether to follow redirects or not
public interface Client {
  Response execute(Request request, Options options) throws IOException;
}

As an example, if we decide to use Apache HTTPClient 5, it would look like:

In [ ]:
public final class ApacheHttp5Client implements Client {
  private final HttpClient client;

  @Override
  public Response execute(Request request, Request.Options options) throws IOException {
    ClassicHttpRequest httpUriRequest;
    try {
      httpUriRequest = toClassicHttpRequest(request, options);
    } catch (final URISyntaxException e) {
      throw new IOException("URL '" + request.url() + "' couldn't be parsed into a URI", e);
    }
    final HttpHost target = HttpHost.create(URI.create(request.url()));
    final HttpClientContext context = configureTimeouts(options);

    final ClassicHttpResponse httpResponse =
        (ClassicHttpResponse) client.execute(target, httpUriRequest, context);
    return toFeignResponse(httpResponse, request);
  }
  
    
  // Other methods  
}

For Apache HTTP Client 5 to be TLS aware, we need to set the `javax.net.ssl.*` properties.

### Options
Lets you specify additional configuration related to the client

In [ ]:
Bank bank = Feign.builder()
        .decoder(new AccountDecoder())
        // Connect timeout, Read timeout and Follow Redirect
        .options(new Request.Options(10, TimeUnit.SECONDS, 60, TimeUnit.SECONDS, true))
        .target(Bank.class, "https://api.examplebank.com");

### Request Interceptor
When we need to change all requests, regardless of their target, we can configure a `RequestInterceptor`.

In [ ]:
Bank bank = Feign.builder()
             .decoder(accountDecoder)
             .requestInterceptor(new BasicAuthRequestInterceptor(username, password))
             .target(Bank.class, "https://api.examplebank.com");

In [ ]:
public interface RequestInterceptor {
  void apply(RequestTemplate template);
}

### ErrorDecoder
Responses where Response.status() is not in the 2xx range are classified as errors, addressed by the `ErrorDecoder`. If we need more control over handling unexpected responses, Feign instances can register a custom `ErrorDecoder` via the builder. All responses that result in an HTTP status not in the 2xx range will trigger the `ErrorDecoder`'s `decode` method. If we want to retry the request again, we can throw a `RetryableException`.

In [ ]:
public class MyErrorDecoder implements ErrorDecoder {
    @Override
    public Exception decode(String methodKey, Response response) {
        return new WebApplicationException(new FeignJaxRsResponse);
    }
}

### Retryer
Feign, by default, will automatically retry `IOExceptions`, regardless of HTTP method, treating them as transient network related exceptions, and any `RetryableException` thrown from an `ErrorDecoder`. To customize this behavior, register a custom `Retryer` instance via the builder.

In [ ]:
public interface Retryer extends Cloneable {
  // if retry is permitted, return (possibly after sleeping). Otherwise propagate the exception
  void continueOrPropagate(RetryableException e);
    
  Retryer clone();
}

## Spring OpenFeign
Lets us configure Feign clients using annotations. Example:

In [ ]:
@SpringBootApplication
@EnableFeignClients
public class AppMain {
    public static void main(String[] args) {
        SpringApplication.run(Application.class, args);
    }
}

@FeignClient("github")
@Path("/repos/{owner}/{repo}/contributors")
interface GitHub {
  @GET
  Response contributors(@PathParam("owner") String owner, @PathParam("repo") String repo);
}

`FeignClientsConfiguration` class contains configuration for Feign components like `Decoder`, `Encoder`, `Contract`, etc. We can configure specific configuration per client by:

In [ ]:
// Do not add @Configuration else beans defined here will become the default bean
@FeignClient(name = "stores", configuration = FooConfiguration.class)
public interface StoreClient {

Host URL, can be specified by setting the url property. In case we want service discovery to provide that, we need to not pass in this property.

In [ ]:
@FeignClient(name = "stores", url = "http://stores.com")
public interface StoreClient {

There may be instance where we need two different clients that point to the same URL. In this case just specifying the name would not be enough. Therefore, we would need to pass in `contextId` which would be used (instead of name) to uniquely identify a client.

In [ ]:
@FeignClient(name = "stores", contextId="firstStore")
public interface StoreClient {

We can also configure Feign clients through properties, for example in `application.yaml`
```yaml
feign:
    client:
        config:
            stores:
                connectTimeout: 5000
                readTimeout: 5000
                    
# Default configuration applying to all clients
feign:
    client:
        config:
            default:
                connectTimeout: 5000
                readTimeout: 5000
```

### Contract
By default Spring provides `SpringMvcContract` bean. To change `Contract` bean, we need to configure a different implementation of `Contract` either through Java or through property definition

In [ ]:
@Bean
public Contract getContract() {
    return new JAXRSContract();
}

```yaml
feign:
    client:
        config:
            default:
                contract: feign.jaxrs.JAXRSContract
```

Property definitions take higher precedence.

### Encoder
By default Spring provides `SpringEncoder` bean. `SpringEncoder` sets null charset for binary content types and UTF-8 for all the other ones. We can modify this behaviour to derive the charset from the `Content-Type` header charset instead by setting the value of `feign.encoder.charset-from-content-type` to true.

### Decoder
Spring sets `ResponseEntityDecoder` as the default `Encoder`

```yaml
feign:
    client:
        config:
            default:
                encoder: com.example.SimpleEncoder
                decoder: com.example.SimpleDecoder
```

### Client
If Spring Cloud LoadBalancer is on the classpath, `FeignBlockingLoadBalancerClient` is used. If none of them is on the classpath, the default feign client is used. If we have Spring Retry in the classpath and `spring.cloud.loadbalancer.retry.enabled` is set to true, `RetryableFeignBlockingLoadBalancerClient` is used instead. Both the implementations make use of service discovery and make load balanced service calls.

Both the above implementations delegate to an actual HTTP Client such as `OkHttpClient` or `ApacheHttpClient` or `ApacheHC5` based on the following properties:
- `feign.okhttp.enabled`
- `feign.httpclient.enabled`
- `feign.httpclient.hc5.enabled`

### ErrorDecoder
Spring does not provider a default bean for `ErrorDecoder`. 

```yaml
feign:
    client:
        config:
            default:
                errorDecoder: com.example.SimpleErrorDecoder
```

### Retryer
A bean of `Retryer.NEVER_RETRY` with the type `Retryer` is created by default, which will disable retrying. This retrying behavior is different from the Feign default one, where it will automatically retry `IOExceptions`, treating them as transient network related exceptions, and any `RetryableException` thrown from an `ErrorDecoder`.